In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

F = 4500 # N
v = 3.05 # m/s
D = 500 # mm
L = 4 # year
t1 = 12 # s
t2 = 60 # s
T1 = 1 # N.mm
T2 = 0.7 # N.mm

## Motor Design $(power, rpm)$

In [2]:
# calculate efficiency, e = eta
e_coupling = 1
e_bearing = 0.99
e_helical_gear = 0.96
e_ch = 0.95
e_sys = e_coupling * e_bearing**3 * e_helical_gear * e_ch
print(e_sys)

P_max = F*v/1000 # calculate max power of the belt conveyor, kW
P_work = P_max*np.sqrt((T1**2*t1 + T2**2*t2)/(t1+t2)) # calculate power given the work load, kW
P_motor = P_work/e_sys # lower limit of the motor's power, kW
print(P_max, P_work, P_motor)

n_belt = v*6e4/np.pi/D # calculate rotational speed of the belt conveyor, rpm
u_ch = 5
u_hg = 5
u_sys = u_ch * u_hg # calculate transmission ratio of the system
n_motor = u_sys * n_belt # calculate theoretical rotational speed of the motor, rpm
print(n_belt, u_sys, n_motor)
P_motor = 18.5 # power based on choice of motor, kW
n_motor = 2930 # rotational speed based on choice of motor, rpm
u_sys = n_motor / n_belt # new transmission ratio
u_ch = u_sys / u_hg # new transmission ratio of ch drive, given u_hg=const
print(u_sys, u_ch)

P_ch = P_max
P_sh2 = P_ch / (e_ch*e_bearing) # power of shaft 2, kW
P_sh1 = P_sh2 / (e_helical_gear*e_bearing) # power of shaft 1, kW
P_motor = P_sh1 / (e_coupling) # power of motor, kW
n_sh1 = n_motor # rotational speed of shaft 1, rpm
n_sh2 = n_sh1 / (u_hg) # rotational speed of shaft 2, rpm
T_motor = 9.55e6 * P_motor / n_motor # torque of motor, N.mm
T_sh1 = 9.55e6 * P_sh1 / n_sh1 # torque of shaft 1, N.mm
T_sh2 = 9.55e6 * P_sh2 / n_sh2 # torque of shaft 2, N.mm
print(np.round([[P_sh2, P_sh1, P_motor],
                [n_motor, n_sh1, n_sh2],
                [T_motor, T_sh1, T_sh2]],2))

0.8849126879999999
13.725 10.407496546960752 11.761043420546766
116.50141834326739 25 2912.535458581685
25.14990840168878 5.029981680337756
[[    14.59     15.35     15.35]
 [  2930.     2930.      586.  ]
 [ 50047.56  50047.56 237825.99]]


## Chain Drive Design

In [3]:
# find z1, z2
z_1 = np.ceil(29 - 2*u_ch)
z_2 = np.ceil(u_ch*z_1) + 1 # eq (5.1), p.80, plus 1 to obtain odd number of teeth

# find k, k_z, k_n
n01 = 600 # ~ n_sh2
k_z, k_n = 25/z_1, n01/n_sh2 # eq (5.3), p.81
k_0, k_a, k_dc, k_bt, k_d, k_c = 1, 1, 1, 1, 1.25, 1.3
k = k_0 * k_a * k_dc * k_bt * k_d * k_c # table 5.6, p.82
P_allowance = P_ch * k * k_z * k_n # equals 30.047 <= 42
P_allowance, p, dc, B = 42, 31.75, 9.55, 27.46 # table 5.5, p.81
d_1 = p/np.sin(np.pi/z_1) # examine table 5.8, p<p_max~31.75
d_2 = p/np.sin(np.pi/z_2) # examine table 5.8, p<p_max~31.75
a_min, a_max, a = 30*p, 50*p, 1000 # 952.5<a<1587.5, choose a = 1000
x = 2*a/p + (z_1+z_2)/2 + (z_2-z_1)**2*p/(4*np.pi**2*a) # eq (5.12), p.85
x_c = np.ceil(x)
a = 0.25*p*(x_c-0.5*(z_2+z_1)+np.sqrt((x_c-0.5*(z_2+z_1))**2-2*((z_2-z_1)/np.pi)**2)) - 0.003*a # eq (5.13), p.85
i = z_1*n_sh2/(15*x) # eq (5.14), p.85; i<[i]=25 at table 5.9, p.85
B_min, d_O, d_l, h_max, b_max, Q, q = 15.88, 7.95, 15.88, 24.2, 38, 56.7, 2.6 # table 5.2, p.78
k_f, k_d = 4, 1.2
v_1 = n_sh2*p*z_1/6e4
F_t, F_v, F_O = 1e3*P_ch/v_1, q*v_1**2, 9.81*k_f*q*a/1e3
s = Q*1e3/(k_d*F_t + F_O + F_v)
s >= 10.3 # table 5.10, p.86
from scipy.interpolate import interp1d
n1 = np.array([1250,1000,900,800,630,500,400,300])
pm = np.array([12.7,15.875,19.5,25.4,31.75,38.1,44.5,50.8])
f = interp1d(n1,pm,kind='cubic')
k_x = 1.15
F_r = k_x*F_t # eq (5.20), p.88
print(np.round([i,v_1,F_t,F_v,F_O,s,F_r, F_O+F_v, F_O+F_v+F_t],2))

[   5.9     5.89 2329.53   90.25  101.92   18.98 2678.96  192.17 2521.7 ]


In [14]:
H1, H2, NFO1, NFO2,m  = 250, 240, 4e6,4e6, 1.5
NHO1, NHO2 = 30*H1**2.4, 30*H2**2.4
NHE1, NHE2 = 60*n_sh1*(T1**3*t1 + T2**3*t2), 60*n_sh2*(T1**3*t1 + T2**3*t2)
NFE1, NFE2 = 60*n_sh1*(T1**6*t1 + T2**6*t2), 60*n_sh2*(T1**6*t1 + T2**6*t2)
KHL1, KHL2 = (NHO1/NHE1)**(1/6), (NHO2/NHE2)**(1/6)
KFL1, KFL2 = (NFO1/NFE1)**(1/6), (NFO2/NFE2)**(1/6)
sigH1, sigH2 = 570/1.1*KHL1, 550/1.1*KHL2
sigF1, sigF2 = 450*KFL1/1.75, 432*KFL2/1.75
sigH = np.average([sigH1,sigH2])
psiba = 0.5
psibd = 0.53*psiba*(u_hg+1)
beta = 15
aw = 43*(u_hg+1)*(T_sh1*1.108/sigH**2/u_hg/0.5)**(1/3)
z1 = 2*aw*np.cos(beta*np.pi/180) / (m*(u_hg+1))
z1 = 21
z2 = u_hg*21
aw = 100
bw = psiba*aw
zmin = 16
y = aw/m-(z1+z2)/2
ky = 1000*y/(z1+z2)
kys = [26,27,28,29,30,31]
kxs = [4.07,4.43,4.76,5.07,5.42,5.76]
f = interp1d(kys,kxs,kind='cubic')
kx = f(ky)
dy = kx*(z1+z2)/1000
print(y,ky,kx,dy)

3.6666666666666714 29.10052910052914 5.1032708861433775 0.6430121316540656


In [8]:
alpha_tw = np.arccos((z1+z2)*m*np.cos(15*np.pi/180)/2/aw)*180/np.pi

alpha, beta = 20*np.pi/180, 20*np.pi/180
alpha_t = np.arctan(np.tan(alpha)/np.cos(beta))
d1 = m*z1/np.cos(beta)
d2 = m*z2/np.cos(beta)
da1 = d1 + 2*m
da2 = d2 + 2*m
df1 = d1 - 2.5*m
df2 = d2 - 2.5*m
db1 = d1*np.cos(alpha_t)
db2 = d2*np.cos(alpha_t)
beta_b = np.arctan(np.cos(alpha_t)*np.tan(beta))
zm, zh = 274, np.sqrt(2*np.cos(beta_b)/np.sin(2*alpha_t))
epbeta = bw*np.sin(beta)/m/np.pi
epalpha = (np.sqrt(da1**2-db1**2)+np.sqrt(da2**2-db2**2)-2*aw*np.sin(alpha_t))/(2*np.pi*m*np.cos(alpha_t)/np.cos(beta))
ze = np.sqrt(1/epalpha)
dw1 = d1
dw2 = d2
v = np.pi*d1*n_sh1/60000
kh, kf, mn = 1.0417*1.0766*1.108, 1.2558*1.1145*1.253, m*np.cos(beta)
sigHc = zm*zh*ze*np.sqrt(2*T_sh1*kh*(u_hg+1)/bw/u_hg/dw1**2)
ye, yb = epalpha**-1, 1-beta*180/np.pi/140
zv1, zv2, yf1, yf2 = z1/np.cos(beta)**3, z2/np.cos(beta)**3, 4.0588, 3.5981
sigF1 = 2*T_sh1*kf*ye*yb*yf1/bw/dw1/mn
sigF2 = sigF1*yf2/yf1

## Shaft design

In [9]:
Ft1 = Ft2 = 2*T_sh1/dw1
Fr1 = Fr2 = Ft1*np.tan(alpha_t)/np.cos(beta)
Fa1 = Fa2 = Ft1*np.tan(beta)
r12, r21 = -dw1/2, dw2/2
hr12, cb12, cq1, hr21, cb21, cq2 = 1,1,1,-1,-1,-1
Fx12 = r12/abs(r12)*cq1*cb12*Ft1
Fx21 = r21/abs(r21)*cq2*cb21*Ft2
Fy12 = -r12/abs(r12)*Ft1*np.tan(alpha_t)/np.cos(beta)
Fy21 = -r21/abs(r21)*Ft2*np.tan(alpha_t)/np.cos(beta)
Fz12 = cq1*cb12*hr12*Ft1*np.tan(beta)
Fz21 = cq2*cb21*hr21*Ft2*np.tan(beta)
Fy22 = F_r*np.cos(210*np.pi/180)
Fx22 = F_r*np.sin(210*np.pi/180)
d1 = (T_sh1/.2/15)**(1/3)
d2 = (T_sh2/.2/30)**(1/3)
bO1,bO2,lm13,lm12,lm22,lm23,k1,k2,k3,hn=15,21,1.5*d1,1.5*d1,1.5*d2,1.5*d2,10,8,15,18
l22 = -(.5*(lm22+bO2)+k3+hn)
l23 = 0.5*(lm23+bO2)+k1+k2
l21 = 2*l23
l12 = -(.5*(lm12+bO1)+k3+hn)
l13 = 0.5*(lm13+bO1)+k1+k2
l11 = 2*l13
print(np.round([l22,l23,l21,l12,l13,l11],2))
print(np.round([Fx12,Fy12,Fz12,Fx21,Fy21,Fz21,Fx22,Fy22],2))

[-69.07  54.07 108.14 -59.66  44.66  89.33]
[-2985.99  1230.79  1086.81  2985.99 -1230.79 -1086.81 -1339.48 -2320.05]


In [13]:
from sympy import *
rA2y, rA2x, rB2y, rB2x, rA1y, rA1x, rB1y, rB1x = symbols('r_A2y, r_A2x, r_B2y, r_B2x, r_A1y, r_A1x, r_B1y, r_B1x')
eq1 = rA2x + rB2x + Fx22 + Fx21
eq2 = -l22*rA2x + (-l22+l23)*Fx21 + (-l22+l21)*rB2x
rA2x,rB2x = list(linsolve([eq1,eq2],[rA2x,rB2x]))[0]
eq1 = Fy22 + rA2y + Fy21 + rB2y
eq2 = -l22*rA2y + (-l22+l23)*Fy21 + (-l22+l21)*rB2y
rA2y,rB2y = list(linsolve([eq1,eq2],[rA2y,rB2y]))[0]
eq1 = rA1x + rB1x + Fx12
eq2 = -l12*rB1x + (-l12+l13)*Fx12 + (-l12+l11)*rA1x
rA1x,rB1x = list(linsolve([eq1,eq2],[rA1x,rB1x]))[0]
eq1 = Fy12 + rA1y + rB1y
eq2 = -l12*rB1y + (-l12+l13)*Fy12 + (-l12+l11)*rA1y
rA1y,rB1y = list(linsolve([eq1,eq2],[rA1y,rB1y]))[0]
np.round([float(rA1x),float(rA1y),float(rB1x),float(rB1y),
         float(rA2x),float(rA2y),float(rB2x),float(rB2y)],2)
M2xA, M2xD = Fx22*-l12, Fx22*(-l12+l23) + rA2x*l23
print(M2xA, M2xD)

-79918.5264686508 -114387.366351581


In [11]:
Fx12

-2985.9885299824223